<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f1b9190ade7bc7823dd28ad8bca4adc994b9484c3db8f82adf2f83a59a5b2a64
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 11:04:37
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -24.59 K (-0.17%)
Current PnL: -22.63 L (-14.83%)
CY Booked + Current PnL: -8.74 L (-5.73%)
-------------------
Total profit:  1.55 L
Total loss:  -24.18 L
-------------------
Total Booked + Current PnL: 18.24 L (14.52%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.04 L (61.5%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.0,0.71,-8.37,9.15,0.02,14636.0,-14605.0,159954.0,143.86,68.0,M-SC,10.48,234.0,-1.00,1.12,39.13,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.0,-2.07,-15.90,19.01,0.09,16112.0,-16020.0,84757.0,96.80,40.0,M-SC,3.53,253.0,-0.99,0.59,11.35,OX40N,NTT,DURABLES
43,ITC,452.0,-0.32,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.94,X40,NTT,FMCG
83,VOLTAS,1530.0,-2.25,6.91,11.96,19.69,24517.0,13248.0,204990.0,-3.56,48.0,X-MC,3.30,78.0,0.54,1.43,14.18,XY25,NTT,AC
53,NESTLEIND,1377.0,-0.01,11.64,8.74,21.40,25898.0,30888.0,296314.0,3.53,52.0,X-LC,7.53,12.0,1.19,2.07,19.05,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,2.94,-2.83,39.62,35.67,121598.0,-8932.0,306910.0,-14.36,62.0,X-LC,5.72,4.0,-0.07,2.14,10.63,X40N,ATH,FMCG
14,CAMPUS,393.00,2.03,-6.16,42.03,33.28,66173.0,-10333.0,157442.0,-24.45,54.0,M-SC,2.93,222.0,-0.16,1.10,22.03,XY24,NTT,FOOTWEAR
7,ATULAUTO,844.00,1.11,-18.42,78.44,45.57,126147.0,-36312.0,160820.0,3478.38,52.0,M-SC,3.96,244.0,-0.29,1.12,14.02,XY24,NTT,AUTO
84,WHIRLPOOL,2270.00,0.96,-13.93,111.36,81.92,110033.0,-15990.0,98808.0,-49.60,25.0,M-SC,2.89,236.0,-0.15,0.69,16.97,XR,NTT,DURABLES
38,INDIAMART,4810.62,0.90,0.73,105.23,106.72,130729.0,896.0,124232.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.27,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.10,-2.69,-26.84,90.83,39.61,68113.0,-27510.0,74990.0,-50.46,36.0,H-SC,21.10,135.0,-0.40,0.52,5.69,AR,ATH,MISC
83,VOLTAS,1530.00,-2.25,6.91,11.96,19.69,24517.0,13248.0,204990.0,-3.56,48.0,X-MC,3.30,78.0,0.54,1.43,14.18,XY25,NTT,AC
78,TTKPRESTIG,770.00,-2.07,-15.90,19.01,0.09,16112.0,-16020.0,84757.0,96.80,40.0,M-SC,3.53,253.0,-0.99,0.59,11.35,OX40N,NTT,DURABLES
67,SIS,528.00,-1.89,-22.32,56.10,21.26,48388.0,-24778.0,86254.0,2037.31,53.0,H-SC,3.76,166.0,-0.51,0.60,16.54,OX40N,NTT,MISC
48,KPIGREEN,730.66,-1.58,-13.14,69.17,46.95,75284.0,-16458.0,108839.0,-19.60,27.0,H-SC,2.27,154.0,-0.22,0.76,36.35,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.90,0.73,105.23,106.72,130729.0,896.0,124232.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.27,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.55,-0.62,36.47,35.63,75960.0,-1293.0,208281.0,-14.53,44.0,H-MC,3.78,119.0,-0.02,1.45,15.95,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.68,-5.20,119.31,107.91,166998.0,-7674.0,139970.0,-22.98,35.0,M-SC,10.68,216.0,-0.05,0.98,0.09,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.71,-8.37,9.15,0.02,14636.0,-14605.0,159954.0,143.86,68.0,M-SC,10.48,234.0,-1.00,1.12,39.13,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-2.07,-15.90,19.01,0.09,16112.0,-16020.0,84757.0,96.80,40.0,M-SC,3.53,253.0,-0.99,0.59,11.35,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.84,-21.65,44.83,13.47,94715.0,-58392.0,211275.0,-67.73,36.0,H-SC,1.62,158.0,-0.62,1.48,7.43,XY24,NTT,PAINTS
67,SIS,528.00,-1.89,-22.32,56.10,21.26,48388.0,-24778.0,86254.0,2037.31,53.0,H-SC,3.76,166.0,-0.51,0.60,16.54,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.11,2.47,68.11,72.27,118261.0,4184.0,173632.0,-11.98,59.0,M-LC,3.68,99.0,0.04,1.21,8.80,XR,NTT,IT
38,INDIAMART,4810.62,0.90,0.73,105.23,106.72,130729.0,896.0,124232.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.27,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.11,2.47,68.11,72.27,118261.0,4184.0,173632.0,-11.98,59.0,M-LC,3.68,99.0,0.04,1.21,8.80,XR,NTT,IT
38,INDIAMART,4810.62,0.90,0.73,105.23,106.72,130729.0,896.0,124232.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.27,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.55,-0.62,36.47,35.63,75960.0,-1293.0,208281.0,-14.53,44.0,H-MC,3.78,119.0,-0.02,1.45,15.95,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.68,-5.20,119.31,107.91,166998.0,-7674.0,139970.0,-22.98,35.0,M-SC,10.68,216.0,-0.05,0.98,0.09,OX40N,ATH,CABLES
1,ABB,7934.00,-0.27,-4.11,51.80,45.57,129952.0,-10747.0,250872.0,-38.36,58.0,H-MC,5.73,120.0,-0.08,1.75,6.75,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.01,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.06,-9.03,26.67,15.23,48606.0,-18097.0,182250.0,-22.11,29.0,X-MC,6.97,66.0,-0.37,1.27,20.30,X40N,NTT,REALTY
9,BAJAJHFL,181.50,0.12,-13.72,72.07,48.45,124672.0,-27519.0,172987.0,-19.87,33.0,X-MC,7.52,64.0,-0.22,1.21,0.50,X40N,ATH,FINANCE
55,PGHH,17907.65,0.16,-4.39,39.90,33.76,76608.0,-8820.0,192000.0,-32.51,34.0,X-MC,3.87,57.0,-0.12,1.34,0.58,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.32,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.94,X40,NTT,FMCG
3,ACC,3906.0,-0.48,-21.78,109.85,64.15,204464.0,-51821.0,186130.0,-54.13,51.0,X-SC,1.54,82.0,-0.25,1.30,4.41,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.48,-11.44,41.72,25.51,68759.0,-21285.0,164810.0,1.14,63.0,H-LC,1.87,49.0,-0.31,1.15,21.45,AR,ATH,ELECTRICAL
15,CAMS,4762.0,-0.41,1.64,22.78,24.79,58314.0,4132.0,255988.0,-3.02,48.0,X-SC,1.87,86.0,0.07,1.79,25.16,X40N,NTT,MISC
76,TMPV,600.0,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.01,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-1.37,-6.20,34.41,26.07,52847.0,-10160.0,153580.0,-30.92,34.0,X-MC,9.57,55.0,-0.19,1.07,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.12,-13.72,72.07,48.45,124672.0,-27519.0,172987.0,-19.87,33.0,X-MC,7.52,64.0,-0.22,1.21,0.50,X40N,ATH,FINANCE
55,PGHH,17907.65,0.16,-4.39,39.90,33.76,76608.0,-8820.0,192000.0,-32.51,34.0,X-MC,3.87,57.0,-0.12,1.34,0.58,X40,ATH,FMCG
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.51,-28.38,99.44,42.84,46298.0,-18447.0,46559.0,-53.17,40.0,X-SC,37.60,83.0,-0.40,0.33,1.31,XY24,NTT,MISC
60,RELAXO,1176.00,-0.06,-46.91,186.90,52.31,144027.0,-68099.0,77061.0,-43.47,38.0,X-SC,4.51,91.0,-0.47,0.54,3.15,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.01,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.05,-6.01,29.36,21.58,38672.0,-8424.0,131715.0,-52.64,36.0,X-MC,6.41,56.0,-0.22,0.92,16.61,X40N,ATH,IT
52,MEDANTA,1486.00,-1.06,3.26,20.16,24.08,26428.0,4140.0,131090.0,-5.17,46.0,X-SC,5.90,89.0,0.16,0.92,23.52,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.09,-14.70,39.83,19.28,117544.0,-50843.0,295113.0,-25.57,58.0,X-LC,7.11,1.0,-0.43,2.06,9.10,X40,ATH,IT
41,INFY,2275.00,-0.31,6.30,45.69,54.87,151965.0,19712.0,332600.0,-17.09,61.0,X-LC,2.48,2.0,0.13,2.32,15.10,X40,BTT,IT
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO
82,VBL,671.64,2.94,-2.83,39.62,35.67,121598.0,-8932.0,306910.0,-14.36,62.0,X-LC,5.72,4.0,-0.07,2.14,10.63,X40N,ATH,FMCG
43,ITC,452.00,-0.32,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.11,-36.77,110.16,32.89,53780.0,-28386.0,48820.0,-702.61,65.0,L-MC,5.75,259.0,-0.53,0.34,34.45,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.21,-12.21,106.95,81.67,88077.0,-11457.0,82353.0,7405.56,54.0,L-SC,19.60,271.0,-0.13,0.58,59.48,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.00,-11.86,58.32,39.54,51999.0,-11997.0,89162.0,-31.03,67.0,M-SC,6.67,220.0,-0.23,0.62,25.73,AR,ATH,AUTO
51,MASFIN,398.61,0.39,-6.43,30.44,22.05,27907.0,-6300.0,91680.0,-19.15,46.0,H-SC,6.94,164.0,-0.23,0.64,33.26,XR,ATH,FINANCE
70,SURYODAY,216.00,0.08,-19.83,53.46,23.03,76750.0,-35506.0,143565.0,55.70,48.0,H-SC,9.57,167.0,-0.46,1.00,42.45,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.71,-8.37,9.15,0.02,14636.0,-14605.0,159954.0,143.86,68.0,M-SC,10.48,234.0,-1.00,1.12,39.13,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.13,0.51,20.34,20.95,51469.0,1276.0,253044.0,-0.49,75.0,X-LC,15.17,31.0,0.02,1.77,36.32,X40,ATH,PAINTS
50,LTIM,7201.88,0.37,10.56,19.09,31.66,51951.0,25987.0,272138.0,6.30,65.0,H-LC,13.54,43.0,0.50,1.90,51.00,X200,ATH,IT
79,UNITDSPR,1644.00,0.55,6.80,13.08,20.77,32707.0,15910.0,250054.0,-2.48,62.0,X-MC,3.79,62.0,0.49,1.75,14.18,X40N,NTT,BREWERIES
40,INDUSINDBK,1800.00,-0.11,-36.77,110.16,32.89,53780.0,-28386.0,48820.0,-702.61,65.0,L-MC,5.75,259.0,-0.53,0.34,34.45,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,44.81
2,50,75.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.53
MC    29.12
LC    25.36
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.17
X40N     12.01
XR        9.54
AR        9.09
XY25      9.08
OX40N     7.63
X200      1.90
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.64
X-LC    20.13
M-SC    11.97
X-SC     8.25
H-MC     4.76
H-LC     3.05
L-SC     1.41
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.87,-4.20,37.90
IT,13.81,-14.35,73.38
FINANCE,9.48,-14.23,62.50
MISC,7.40,-22.78,73.08
PAINTS,5.97,-8.16,24.23
ELECTRICAL,5.97,-9.65,49.82
INSURANCE,4.41,-0.26,35.59
PHARMA,3.86,-0.88,33.16
AUTO,3.37,-18.84,69.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3050433.0,21
XR,1298163.0,13
AR,1269344.0,10
X40,1008206.0,14
X40N,729718.0,9
OX40N,700991.0,10
XY25,339645.0,6
SR,280427.0,2
MH,75284.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3560096.0,25
M-SC,1347199.0,15
X-MC,1239629.0,16
X-LC,881021.0,11
X-SC,740557.0,8
H-MC,395749.0,3
L-SC,259778.0,3
H-LC,120710.0,2
M-LC,118261.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213829.0      6
           AR         873939.0      5
           XR         795467.0      7
M-SC       XY24       775747.0      6
X-MC       X40        448920.0      7
X-LC       X40        390951.0      5
X-MC       XY24       365721.0      3
H-SC       OX40N      321150.0      4
X-SC       X40N       295032.0      3
M-SC       OX40N      291757.0      5
H-SC       SR         280427.0      2
X-SC       XY24       277190.0      3
X-MC       X40N       244657.0      4
X-LC       XY24       228109.0      2
H-MC       AR         205912.0      2
X-LC       X40N       190029.0      2
H-MC       XY24       189837.0      1
X-MC       XY25       180331.0      2
L-SC       XR         171694.0      2
X-SC       X40        168335.0      2
M-SC       XR         158961.0      2
           AR         120734.0      2
M-LC       XR         118261.0      1
L-SC       OX40N       88084.0      1
H-SC       MH          75284.0      1
X-LC       XY25        71932.0      2
H-LC       AR          68759.0      1
L-MC       XR          53780.0      1
H-LC       X200        51951.0      1
M-MC       XY25        47335.0      1
L-LC       XY25        40047.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
